<a href="https://colab.research.google.com/github/Anil951/Early-detection-of-mental-health/blob/main/implementation_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STAGE 2: SUDDEN CHANGES IN SCORES, ATTENDANCE, AND TEACHER REMARKS

# SIGNIFICANT DECREASES IN A STUDENT'S SCORES, ATTENDANCE,
# AND TEACHER REMARKS FROM ONE TERM TO THE NEXT MAY INDICATE THAT THEY COULD BE EXPERIENCING MENTAL HEALTH CHALLENGES.

# DISCLAIMER: This is an assumption based on observed patterns and may not necessarily reflect the actual mental health status of the student.

In [ ]:
# do OCR(pytesseract) and extract data from images and store in DATAFRAMES
# and see percent of decrease in SCORE, ATTENDENCE, REMARKS SENTIMENT

In [ ]:
# for PROTOTYPE/DEMO purpose we are taking only demo score cards (found in data/demo score cards/)

# DISCLAIMER: these score cards are generated ones (refer : generate_scorecards_images.ipynb)

In [1]:
!pip install pytesseract

In [2]:
!apt-get update
!apt-get install -y tesseract-ocr

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
!pip install pytesseract

In [ ]:
import pandas as pd
import re
from PIL import Image
import pytesseract

# Open the image
image = Image.open('/content/report_card_class10.jpg')

# Extract text from the image using Tesseract OCR
text = pytesseract.image_to_string(image).lower()


# PREPROCESSING

def clean(subject):
    return re.sub(r"[^0-9a-zA-Z\s]", "", subject).strip()

# Split the text into lines and remove lines containing a colon
lines = [line.strip() for line in text.split('\n') if ':' not in line and 'total' not in line]
for i in lines:
  if(len(i)==0):
      lines.remove(i)

# Apply cleaning to each line and remove any empty lines
lines = [clean(line) for line in lines if clean(line)]

print(lines)

['term1', 'subject', 'math', 'sclence', 'english', 'history', 'physical education', 'score', '90', '85', '88', '80', '95', 'attendance', '92', '88', '90', '93', '97', 'remarks', 'consistently performs well shows strong skils', 'active in discussions good understanding of concepts', 'strong kills participates actively in group work', 'engaged in class discussions needs improvement', 'high energy and excellent team player', 'term2', 'subject', 'math', 'sclence', 'english', 'history', 'physical education', 'score', '85', '78', '80', '72', '90', 'attendance', '89', '84', '85', '88', '92', 'remarks', 'shows occasional struggles  needs extra practice', 'lacks motivation in class needs more focus', 'reduced participation distracted in class', 'difficulties in remembering needs to engage more', 'should improve fitness level', 'term3', 'subject', 'math', 'sclence', 'english', 'history', 'physical education', 'score', '75', '70', '72', '65', '80', 'attendance', '83', '75', '78', '80', '85', 'rem

In [ ]:
# Extracting term data into dataframes
terms = {}
data=lines
i = 0
while i < len(data):
    if 'term' in data[i]:
        # Initialize term info
        term_name = data[i]
        terms[term_name] = {}
        i += 1
        # Continue until next term or end of list
        while i < len(data) and 'term' not in data[i]:
            # Column name and values
            col_name = data[i]
            col_values = data[i + 1:i + 6]
            terms[term_name][col_name] = col_values
            i += 6  # Move to next column name
    else:
        i += 1  # Skip any stray data

# Creating dataframes for each term
dfs = {term: pd.DataFrame(data) for term, data in terms.items()}
dfs

{'term1':               subject score attendance  \
 0                math    90         92   
 1             sclence    85         88   
 2             english    88         90   
 3             history    80         93   
 4  physical education    95         97   
 
                                              remarks  
 0      consistently performs well shows strong skils  
 1  active in discussions good understanding of co...  
 2   strong kills participates actively in group work  
 3     engaged in class discussions needs improvement  
 4              high energy and excellent team player  ,
 'term2':               subject score attendance  \
 0                math    85         89   
 1             sclence    78         84   
 2             english    80         85   
 3             history    72         88   
 4  physical education    90         92   
 
                                             remarks  
 0  shows occasional struggles  needs extra practice  
 1        lacks

In [ ]:
dfs['term3']

,subject,score,attendance,remarks
0,math,75,83,struggles with advanced topics seems frustrated
1,sclence,70,75,no interest in class lab work is neglected
2,english,72,78,very little participation often distracted
3,history,65,80,limited engagement affecting performance
4,physical education,80,85,results decreased tired uninterested


In [ ]:
# calculating the decrease in percent of SCORE, ATTENDENCE, REMARKS sentiment  form term to term

In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download the VADER lexicon for sentiment analysis
nltk.download("vader_lexicon")
sid = SentimentIntensityAnalyzer()

# Term-wise data as dictionaries of DataFrames
term_data = dfs

# Convert scores and attendance to numeric values and analyze sentiments
for term, df in term_data.items():
    df['score'] = pd.to_numeric(df['score'])
    df['attendance'] = pd.to_numeric(df['attendance'])
    df['senti'] = df['remarks'].apply(lambda x: sid.polarity_scores(x)['compound'])

# Calculate the average score, attendance, and sentiment for each term
avg_term1 = term_data['term1'][['score', 'attendance', 'senti']].mean()
avg_term3 = term_data['term3'][['score', 'attendance', 'senti']].mean()

# Calculate overall percentage decrease from term1 to term3
score_decrease = ((avg_term1['score'] - avg_term3['score']) / avg_term1['score']) * 100
attendance_decrease = ((avg_term1['attendance'] - avg_term3['attendance']) / avg_term1['attendance']) * 100
senti_decrease = avg_term1['senti'] - avg_term3['senti']

# Compile the results into a DataFrame
overall_decrease = pd.DataFrame({
    'Metric': ['Score', 'Attendance', 'Sentiment'],
    'Decrease (%)': [score_decrease, attendance_decrease, senti_decrease * 100]
})

print(overall_decrease)

# Determine overall likelihood of mental health issues
if score_decrease > 15 or attendance_decrease > 15 or senti_decrease < -0.2:
    print("There is a high likelihood of mental health issues based on the observed decreases.")
else:
    print("There is a low likelihood of mental health issues based on the observed decreases.")

       Metric  Decrease (%)
0       Score     17.351598
1  Attendance     12.826087
2   Sentiment     88.694000
There is a high likelihood of mental health issues based on the observed decreases.


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
